In [1]:
from KC_methods import calc_KC76
import pandas as pd
import numpy as np
from logistic_maps import generate_logistic

In [2]:
def discretize(arr):
    return [1 if num >= 0.5 else 0 for num in arr]

def to_bin(arr):
    return ''.join(list(map(str,arr)))

def f(x,r):
    return r * x * (1-x)

def logistic_map(x_0,r,iterations,epsilon=0,e_freq=0):
    res = []
    cur = x_0
    kicks = np.random.rand(iterations) < e_freq
    epsilons = np.random.uniform(-epsilon,epsilon,size=iterations)
    for i,is_kick in enumerate(kicks):
        cur = r * cur * (1-cur)

        # When kick occurs
        if is_kick:
            added_eps = epsilons[i]
            # Keep sampling for epsilon until we get a valid value
            while  0>= cur + added_eps or cur + added_eps>=1:
                added_eps = np.random.uniform(-epsilon,epsilon)
            cur += added_eps
        assert 0<= cur <=1, f"Cur os {cur}"
        res.append(cur)
    return res


In [ ]:
from functools import cache

SAMPLES = 10**6

def generate_patterns(iterations,epsilon=0,e_freq=0,r=(0,4)):

    '''Return list of patterns  from random `r` and `x_0`. patterns are of length `iterations`
        If r parameter is provided it will use that as the initial value of r, otherwise it will be random.
    
    '''
    patterns = [''] *(SAMPLES) 
    print(r)
    for i in range(SAMPLES):
        rr = np.random.uniform(r[0],r[1])
        x_0 = np.random.rand()
        mapp = logistic_map(x_0,rr,iterations,epsilon=epsilon,e_freq=e_freq)
        patterns[i] = to_bin(discretize(mapp))
    return patterns

@cache
def generate_logistic(iterations,start_iteration=0,epsilon=0,e_frequency=0,r=(0,4)):
    ''' Wrapper function for logicstic map generation with kick.

    Parameters:
        `iterations`: the number of iterations that will be taken into account, this will determine the length of the pattern.
        `start_iteration`: after what iteration to start observing the pattern. defaults to `0`.
        `epsilon`: The magnitude of the random kick applied, when 0.5 is passed, the kick will be between -0.5,0.5. default to 0
        `e_frequency`: Nonnegative number determining the frequency of the kick. defaults to 0.
                       If the number is in the range [0,1), this is a probability.
    '''
                       # Otherwise it represents the frequency of the kick (1 means every time, 2 means every two time [nokick,kick,nokick,kick]) NOT IMPLEMENTED
    patterns = generate_patterns(start_iteration+iterations,epsilon=epsilon,e_freq=e_frequency,r=r)
    patterns = [ pattern[start_iteration:] for pattern in patterns]
    return patterns

In [9]:
patterns = generate_logistic(25,0,0,0,(0,4))

(0, 4)


In [ ]:
ar = []
for start in (0,3,3.57,4):
    patterns = generate_logistic(25,0,0,0,(start,4))
    ar.append(patterns)


(0, 4)
(3, 4)


In [12]:
import matplotlib.pyplot as plt
from KC_methods import calc_KC76

for patterns in ar:
    df = pd.DataFrame()
    df["x"] = patterns
    df["K"] = df["x"].apply(calc_KC76)
    df["scaled"] = len(patterns[0]) * ( df["K"]-df["K"].min() ) / ( df["K"].max()-df["K"].min() )
    fig = df["scaled"].hist(density=True)
    plt.yscale("log")
    plt.ylabel("Frequency",fontsize=15)
    plt.xlabel(r'$\mathit{\tilde{K}(x)}$',fontsize=15)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.grid(False)
    plt.tight_layout()
    plt.savefig(f"ttt/hist_scaled_{start}.png")
    plt.clf()

(0, 4)
(3, 4)


/tmp/ipykernel_16371/2350989353.py:17: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


(3.57, 4)


/tmp/ipykernel_16371/2350989353.py:17: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


(4, 4)


/tmp/ipykernel_16371/2350989353.py:17: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


<Figure size 640x480 with 0 Axes>